# Building a Music Catalog with MongoDB
--------------------------------------------------------------------------

This tutorial makes use of the _Musicbrainz_ database: [https://musicbrainz.org/](https://musicbrainz.org/)

In [7]:
#!pip install musicbrainzngs
#!pip install pymongo

In [1]:
import os
import time
import pprint
import datetime
from pymongo import MongoClient, IndexModel, ASCENDING, DESCENDING
import musicbrainzngs
from musicbrainzngs import *

In [2]:
# Config musicbrainz
# https://musicbrainz.org/doc/Development/XML_Web_Service/Version_1

# musicbrainzngs.set_useragent(app="coffeecode", version=1, contact=None)

In [3]:
# Connect to MongoDB

client = MongoClient('localhost', 27017)

## CRUD Operations

In [4]:
# Create a database and add some documents

db = client.crud

In [6]:
# Create a collection

db.drop_collection('simple')
simple = db.simpledocs

In [7]:
# Create a document and insert/add into collection

aDoc = {"author": "Jon", "text": "A MongoDB demo", "tags": ["mongodb", "pymongo"], "date": datetime.datetime.utcnow()}
simple.insert_one(aDoc)

In [8]:
# Add many docs at once - notice they don't all have the same structure

doc2 = {"author": "Waldo", "text": "Where is everybody?", "friends": ["Hal", "Tammy"], "date": datetime.datetime.utcnow()}
doc3 = {"org": "NASA", "text": "Where is everybody?", "tags": ["Fermi", "SETI"], "date": datetime.datetime.utcnow()}
doc4 = {"author": "Hal", "worry": "Are you there?", "date": datetime.datetime.utcnow()}

posts = [doc2, doc3, doc4]

result = simple.insert_many(posts)

In [9]:
result.inserted_ids

[ObjectId('5c09cc68c9efcc2110f5cb6a'),
 ObjectId('5c09cc68c9efcc2110f5cb6b'),
 ObjectId('5c09cc68c9efcc2110f5cb6c')]

In [10]:
# Retrieve using find_one() and find()

findDoc = simple.find_one()
pprint.pprint(findDoc)

{'_id': ObjectId('5c09cc64c9efcc2110f5cb69'),
 'author': 'Jon',
 'date': datetime.datetime(2018, 12, 7, 1, 27, 0, 486000),
 'tags': ['mongodb', 'pymongo'],
 'text': 'A MongoDB demo'}


In [11]:
docs = simple.find()
for d in docs:
    pprint.pprint(d)

{'_id': ObjectId('5c09cc64c9efcc2110f5cb69'),
 'author': 'Jon',
 'date': datetime.datetime(2018, 12, 7, 1, 27, 0, 486000),
 'tags': ['mongodb', 'pymongo'],
 'text': 'A MongoDB demo'}
{'_id': ObjectId('5c09cc68c9efcc2110f5cb6a'),
 'author': 'Waldo',
 'date': datetime.datetime(2018, 12, 7, 1, 27, 4, 87000),
 'friends': ['Hal', 'Tammy'],
 'text': 'Where is everybody?'}
{'_id': ObjectId('5c09cc68c9efcc2110f5cb6b'),
 'date': datetime.datetime(2018, 12, 7, 1, 27, 4, 87000),
 'org': 'NASA',
 'tags': ['Fermi', 'SETI'],
 'text': 'Where is everybody?'}
{'_id': ObjectId('5c09cc68c9efcc2110f5cb6c'),
 'author': 'Hal',
 'date': datetime.datetime(2018, 12, 7, 1, 27, 4, 87000),
 'worry': 'Are you there?'}


In [12]:
findDoc = simple.find_one({"author": "Hal"})
pprint.pprint(findDoc)

{'_id': ObjectId('5c09cc68c9efcc2110f5cb6c'),
 'author': 'Hal',
 'date': datetime.datetime(2018, 12, 7, 1, 27, 4, 87000),
 'worry': 'Are you there?'}


In [13]:
docs = simple.find({"tags": {"$exists": 'True'}})
for d in docs:
    pprint.pprint(d)

{'_id': ObjectId('5c09cc64c9efcc2110f5cb69'),
 'author': 'Jon',
 'date': datetime.datetime(2018, 12, 7, 1, 27, 0, 486000),
 'tags': ['mongodb', 'pymongo'],
 'text': 'A MongoDB demo'}
{'_id': ObjectId('5c09cc68c9efcc2110f5cb6b'),
 'date': datetime.datetime(2018, 12, 7, 1, 27, 4, 87000),
 'org': 'NASA',
 'tags': ['Fermi', 'SETI'],
 'text': 'Where is everybody?'}


In [14]:
# Update

update = simple.update_one({"author": "Jon"}, {'$set': {'author': 'Hal'}})

In [15]:
update = simple.update_many({"author": "Hal"}, { '$addToSet': { 'tags': 'worried'} })

In [16]:
docs = simple.find()
for d in docs:
    pprint.pprint(d)

{'_id': ObjectId('5c09cc64c9efcc2110f5cb69'),
 'author': 'Hal',
 'date': datetime.datetime(2018, 12, 7, 1, 27, 0, 486000),
 'tags': ['mongodb', 'pymongo', 'worried'],
 'text': 'A MongoDB demo'}
{'_id': ObjectId('5c09cc68c9efcc2110f5cb6a'),
 'author': 'Waldo',
 'date': datetime.datetime(2018, 12, 7, 1, 27, 4, 87000),
 'friends': ['Hal', 'Tammy'],
 'text': 'Where is everybody?'}
{'_id': ObjectId('5c09cc68c9efcc2110f5cb6b'),
 'date': datetime.datetime(2018, 12, 7, 1, 27, 4, 87000),
 'org': 'NASA',
 'tags': ['Fermi', 'SETI'],
 'text': 'Where is everybody?'}
{'_id': ObjectId('5c09cc68c9efcc2110f5cb6c'),
 'author': 'Hal',
 'date': datetime.datetime(2018, 12, 7, 1, 27, 4, 87000),
 'tags': ['worried'],
 'worry': 'Are you there?'}


In [17]:
# Delete

delete = simple.delete_one({'tags': 'Fermi'})

In [18]:
docs = simple.find()
for d in docs:
    pprint.pprint(d)

{'_id': ObjectId('5c09cc64c9efcc2110f5cb69'),
 'author': 'Hal',
 'date': datetime.datetime(2018, 12, 7, 1, 27, 0, 486000),
 'tags': ['mongodb', 'pymongo', 'worried'],
 'text': 'A MongoDB demo'}
{'_id': ObjectId('5c09cc68c9efcc2110f5cb6a'),
 'author': 'Waldo',
 'date': datetime.datetime(2018, 12, 7, 1, 27, 4, 87000),
 'friends': ['Hal', 'Tammy'],
 'text': 'Where is everybody?'}
{'_id': ObjectId('5c09cc68c9efcc2110f5cb6c'),
 'author': 'Hal',
 'date': datetime.datetime(2018, 12, 7, 1, 27, 4, 87000),
 'tags': ['worried'],
 'worry': 'Are you there?'}


In [19]:
delete = simple.delete_many({'author': 'Hal'})

In [20]:
docs = simple.find()
for d in docs:
    pprint.pprint(d)

{'_id': ObjectId('5c09cc68c9efcc2110f5cb6a'),
 'author': 'Waldo',
 'date': datetime.datetime(2018, 12, 7, 1, 27, 4, 87000),
 'friends': ['Hal', 'Tammy'],
 'text': 'Where is everybody?'}


In [21]:
db.drop_collection('simple')

{'ok': 0.0,
 'errmsg': 'ns not found',
 'code': 26,
 'codeName': 'NamespaceNotFound'}

In [22]:
client.database_names()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  """Entry point for launching an IPython kernel.


['admin', 'config', 'crud', 'jazz_catalog', 'local']

In [23]:
client.drop_database('crud')

In [24]:
client.database_names()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  """Entry point for launching an IPython kernel.


['admin', 'config', 'jazz_catalog', 'local']

In [25]:
# Create db

db = client.jazz_catalog

In [51]:

# Create three collections
# Note that due to network limitations we will use only a subset of available Musicbrainz metadata

#db.drop_collection('artists')
artists = db.artists

#db.drop_collection('releases')
releases = db.releases


```
# search for an artist

seedArtist = musicbrainzngs.search_artists("miles davis")
```

In [26]:
#for artist in seedArtist['artist-list']:
#    print("{name}, {id}".format(name=artist['name'], id=artist['id']))

```
# Create indexes for mongodb collections
# on ID field, enforce uniqueness constraint

db.artists.create_index('id', name='artist_id', unique=True)
db.releases.create_index('id', name='release_id', unique=True)
```

In [16]:
# Get artist info for top two
# Insert into db.artists

#for artist in seedArtist['artist-list'][:2]:
#    mbid = artist['id']
#    artist_info = musicbrainzngs.get_artist_by_id(mbid)
#    pprint.pprint(artist_info)

{'artist': {'area': {'id': '489ce91b-6658-3307-9877-795b68554c98',
                     'iso-3166-1-code-list': ['US'],
                     'name': 'United States',
                     'sort-name': 'United States'},
            'begin-area': {'id': '39e1719e-2604-4b59-b698-dab7caf85b33',
                           'name': 'Alton',
                           'sort-name': 'Alton'},
            'country': 'US',
            'disambiguation': 'jazz trumpeter, bandleader, songwriter',
            'end-area': {'id': 'dbacf2e3-7e3e-4cee-8804-999b109285fa',
                         'name': 'Santa Monica',
                         'sort-name': 'Santa Monica'},
            'gender': 'Male',
            'id': '561d854a-6a28-4aa7-8c99-323e6ce46c2a',
            'ipi': '00007619785',
            'ipi-list': ['00007619785'],
            'isni-list': ['000000012144707X'],
            'life-span': {'begin': '1926-05-26',
                          'end': '1991-09-28',
                          'ended'

```
for artist in seedArtist['artist-list'][:2]:
    mbid = artist['id']
    artist_info = musicbrainzngs.get_artist_by_id(mbid)
    artists.insert_one(artist_info['artist'])
```

In [29]:
# Try again to check uniqueness constraint

#for artist in seedArtist['artist-list'][:2]:
#    mbid = artist['id']
#    artist_info = musicbrainzngs.get_artist_by_id(mbid)
#    try:
#        artists.insert_one(artist_info['artist'])
#    except Exception as e:
#        print(e)

In [30]:
db.artists.count()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


1504

In [20]:
#for a in artists.find():
#    pprint.pprint(a)

{'_id': ObjectId('5c086ce6c9efcc1303986738'),
 'area': {'id': '489ce91b-6658-3307-9877-795b68554c98',
          'iso-3166-1-code-list': ['US'],
          'name': 'United States',
          'sort-name': 'United States'},
 'begin-area': {'id': '39e1719e-2604-4b59-b698-dab7caf85b33',
                'name': 'Alton',
                'sort-name': 'Alton'},
 'country': 'US',
 'disambiguation': 'jazz trumpeter, bandleader, songwriter',
 'end-area': {'id': 'dbacf2e3-7e3e-4cee-8804-999b109285fa',
              'name': 'Santa Monica',
              'sort-name': 'Santa Monica'},
 'gender': 'Male',
 'id': '561d854a-6a28-4aa7-8c99-323e6ce46c2a',
 'ipi': '00007619785',
 'ipi-list': ['00007619785'],
 'isni-list': ['000000012144707X'],
 'life-span': {'begin': '1926-05-26', 'end': '1991-09-28', 'ended': 'true'},
 'name': 'Miles Davis',
 'sort-name': 'Davis, Miles',
 'type': 'Person'}
{'_id': ObjectId('5c086ce7c9efcc1303986739'),
 'area': {'id': '489ce91b-6658-3307-9877-795b68554c98',
          'iso-316

```
# Get the first set of releases with 'Official' status
# (keep result sets to default 25 to minimize network impact)
# Make a second call for complete track info and
# Insert response into mongodb releases collection

# ADD BACK 'artist-credits' include

for a in artists.find():
    artist_id = a['id']
    release_search = musicbrainzngs.get_artist_by_id(artist_id, includes=['releases',  'release-rels', 'label-rels'])
    release_list = release_search['artist']['release-list']
    for r in release_list:
        if 'status' in r.keys():
            if r['status'] == "Official":
                release_id = r['id']
                release_info = musicbrainzngs.get_release_by_id(release_id,
                                                                includes=['artist-credits', 'recordings',
                                                                          'recording-level-rels', 'artist-rels'])
                try:
                    releases.insert_one(release_info['release'])
                except Exception as e:
                    print(e)
```

In [31]:
db.releases.count()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


889

```
# so now get all artists info for all tracks on all releases

artist_ids = set()
artist_count = 0

for r in releases.find():
    if 'medium-list' in r:
        media = r['medium-list']
        for m in media:
            if 'track-list' in m:
                tracks = m['track-list']
                for each in tracks:
                    if 'recording' in each.keys():
                        if 'artist-relation-list' in each['recording'].keys():
                            artist_list = each['recording']['artist-relation-list']
                            for a in artist_list:
                                artist_count += 1
                                artist_id = a['artist']['id']
                                artist_ids.add(artist_id)
```

In [33]:
#artist_count

In [35]:
#len(artist_ids)

In [36]:
#current_artist_ids = set(artists.distinct('id'))
#new_artist_search_ids = artist_ids - current_artist_ids

In [37]:
#len(new_artist_search_ids)

```
for a_id in list(new_artist_search_ids):
    artist_info = musicbrainzngs.get_artist_by_id(a_id)
    try:
        artists.insert_one(artist_info['artist'])
    except Exception as e:
        print(e)
```

In [39]:
#artists.count()

```
# now redo release search for all new artists - there may be some duplicates

# Get the first set of releases with 'Official' status
# (keep result sets to default 25 to minimize network impact)
# Make a second call for complete track info and
# Insert response into mongodb releases collection

for a_id in list(new_artist_search_ids):
    release_search = musicbrainzngs.get_artist_by_id(a_id, includes=['releases',  'release-rels', 'label-rels'])
    release_list = release_search['artist']['release-list']
    for r in release_list:
        if 'status' in r.keys():
            if r['status'] == "Official":
                release_id = r['id']
                release_info = musicbrainzngs.get_release_by_id(release_id,
                                                                includes=['artist-credits', 'recordings', 
                                                                          'recording-level-rels', 'artist-rels'])
                try:
                    releases.insert_one(release_info['release'])
                except Exception as e:
                    print(e)
```

In [40]:
db.releases.count()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


889

In [42]:
a_release = db.releases.find_one()

In [83]:
pprint.pprint(a_release)

{'_id': ObjectId('5c086cebc9efcc130398673c'),
 'artist-credit': [{'artist': {'disambiguation': 'jazz trumpeter, bandleader, '
                                                 'songwriter',
                               'id': '561d854a-6a28-4aa7-8c99-323e6ce46c2a',
                               'name': 'Miles Davis',
                               'sort-name': 'Davis, Miles'}}],
 'artist-credit-phrase': 'Miles Davis',
 'country': 'US',
 'cover-art-archive': {'artwork': 'true',
                       'back': 'false',
                       'count': '1',
                       'front': 'true'},
 'date': '1954',
 'id': '16ed7b47-e00c-499d-8866-65e660f4c78f',
 'medium-count': 1,
 'medium-list': [{'format': '12" Vinyl',
                  'position': '1',
                  'track-count': 6,
                  'track-list': [{'artist-credit': [{'artist': {'disambiguation': 'jazz '
                                                                                  'trumpeter, '
                 

```
artist_ids = set()
artist_count = 0

for r in releases.find():
    if 'medium-list' in r:
        media = r['medium-list']
        for m in media:
            if 'track-list' in m:
                tracks = m['track-list']
                for each in tracks:
                    if 'recording' in each.keys():
                        if 'artist-relation-list' in each['recording'].keys():
                            artist_list = each['recording']['artist-relation-list']
                            for a in artist_list:
                                artist_count += 1
                                artist_id = a['artist']['id']
                                artist_ids.add(artist_id)
```

In [44]:
#artist_count

In [45]:
#len(artist_ids)

In [46]:
#current_artist_ids = set(artists.distinct('id'))
#new_artist_search_ids = artist_ids - current_artist_ids

In [47]:
#len(new_artist_search_ids)

```
for a_id in list(new_artist_search_ids):
    artist_info = musicbrainzngs.get_artist_by_id(a_id)
    try:
        artists.insert_one(artist_info['artist'])
    except Exception as e:
        print(e)
```

In [48]:
db.artists.count()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


1504

In [49]:
db.releases.count()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


889

## Aggregations

In [52]:
# https://info-mongodb-com.s3.amazonaws.com/ReferenceCards15-PDF.pdf

# pseduo-SQL
'''
SELECT type, count(type)
FROM artists
GROUP BY type
'''

pipeline = [{"$group": {"_id": "$type", "count":{"$sum":1}}}]
artist_types = list(artists.aggregate(pipeline, allowDiskUse=True))
artist_types

[{'_id': 'Person', 'count': 1454},
 {'_id': 'Group', 'count': 25},
 {'_id': 'Orchestra', 'count': 7},
 {'_id': None, 'count': 17},
 {'_id': 'Other', 'count': 1}]

In [53]:
# pseudo-SQL
# requires an artist table and a birthplace table

'''
SELECT begin-area.name, count(begin-area.name)
FROM begin-area
INNER JOIN artist ON begin-area.id == artist.begin-area-id
GROUP BY begin-area
'''

pipeline = [{"$group": {"_id": "$begin-area.name", "count":{"$sum":1}}},
           {"$sort": {"count":-1}},
           {"$limit": 10 }]
artist_birthplaces = list(artists.aggregate(pipeline, allowDiskUse=True))
artist_birthplaces

[{'_id': None, 'count': 640},
 {'_id': 'New York', 'count': 89},
 {'_id': 'Philadelphia', 'count': 48},
 {'_id': 'Chicago', 'count': 29},
 {'_id': 'Brooklyn', 'count': 27},
 {'_id': 'Los Angeles', 'count': 24},
 {'_id': 'Detroit', 'count': 22},
 {'_id': 'Pittsburgh', 'count': 20},
 {'_id': 'Kansas City', 'count': 12},
 {'_id': 'Newark', 'count': 11}]

In [54]:
# pseudo-SQL

'''
SELECT begin-area.name, count(begin-area.name) as c, artist.name
FROM begin-area
INNER JOIN artist ON begin-area.id == artist.begin-area-id
GROUP BY begin-area.name, artist.name
ORDER BY c DESC
'''

# Exclude artists where begin-area field doesn't exist

pipeline = [{"$match": {"begin-area.name": {"$exists": 'True'}}},
            {"$group": {"_id": {"area": "$begin-area.name", "artist": "$name"}, "artistCount":{"$sum":1}}},
            {"$group": {"_id": "$_id.area", "artists": {"$push": {"artist": "$_id.artist"}}, "count":{"$sum":"$artistCount"}}},
            {"$sort": {"count":-1}},
            {"$limit": 2 }]
artist_birthplaces = list(artists.aggregate(pipeline, allowDiskUse=True))
artist_birthplaces

[{'_id': 'New York',
  'artists': [{'artist': 'Charlie Fowlkes'},
   {'artist': 'George Duvivier'},
   {'artist': 'Art Taylor'},
   {'artist': 'Herb Bushler'},
   {'artist': 'Marc Cary'},
   {'artist': 'David Friedman'},
   {'artist': 'David Greene'},
   {'artist': 'Kenny Berger'},
   {'artist': 'Steve Lacy'},
   {'artist': 'Chuck Wayne'},
   {'artist': 'Lenny Castro'},
   {'artist': 'Bill Grauer, Jr.'},
   {'artist': 'Tiny Kahn'},
   {'artist': 'George Barrow'},
   {'artist': 'LaMont Johnson'},
   {'artist': 'Chauncey Welsch'},
   {'artist': 'Eddie Henderson'},
   {'artist': 'Steve Grossman'},
   {'artist': 'Lawrence Dutton'},
   {'artist': 'Al Gafa'},
   {'artist': 'Ronnie Cuber'},
   {'artist': 'Julian Euell'},
   {'artist': 'Lew Soloff'},
   {'artist': 'Frank Rehak'},
   {'artist': 'Bernie Privin'},
   {'artist': 'Joe Shulman'},
   {'artist': 'Dave Kurtzer'},
   {'artist': 'Willie Jones'},
   {'artist': 'Helen Merrill'},
   {'artist': 'Jackie McLean'},
   {'artist': 'Adam Nussbaum'

In [55]:
# single artist discography

pipeline = [{"$match": {'artist-credit-phrase': 'Miles Davis'}},
            {"$group": {"_id": {"artist": "$artist-credit-phrase"}, "release": {"$push": {"title": "$title", "year": "$date"}}}},
            {"$sort": {"year":1, "title":1}}]
discography = list(releases.aggregate(pipeline, allowDiskUse=True))
discography

[{'_id': {'artist': 'Miles Davis'},
  'release': [{'title': 'Miles Davis, Volume 3', 'year': '1954'},
   {'title': 'Collectors’ Items', 'year': '1956'},
   {'title': 'Birth of the Cool', 'year': '1957-02'},
   {'title': "'Round About Midnight", 'year': '1957-03-18'},
   {'title': 'Miles Davis and Horns', 'year': '1956'},
   {'title': 'Blue Moods', 'year': '1955'},
   {'title': 'Miles Ahead', 'year': '1957'},
   {'title': 'The Compositions of Al Cohn', 'year': '1953'},
   {'title': 'Budo / Move', 'year': '1949-03'},
   {'title': 'Miles Davis, Volume 1', 'year': '1953'},
   {'title': 'Miles Davis, Volume 2', 'year': '1956'},
   {'title': "'Round About Midnight", 'year': '1957-03-04'},
   {'title': 'Young Man With a Horn', 'year': '1952'},
   {'title': 'Blue Period', 'year': '1953'},
   {'title': 'Miles Davis Quartet', 'year': '1954'},
   {'title': 'Milestones', 'year': '1958'},
   {'title': 'The New Sounds', 'year': '1951-10-05'},
   {'title': "Bags' Groove", 'year': '1957'},
   {'title'

In [56]:
# complete collection discography

pipeline = [{"$match": {'artist-credit-phrase': {"$exists": 'True'}}},
            {"$group": {"_id": {"artist": "$artist-credit-phrase"}, "release": {"$push": {"title": "$title", "year": "$date"}}}},
            {"$sort": {"artist":1, "year":1, "title":1}},
            {"$limit": 10 }]
discography = list(releases.aggregate(pipeline, allowDiskUse=True))
discography

[{'_id': {'artist': 'Gerry Mulligan & Bob Brookmeyer'},
  'release': [{'title': 'Jazz Concerto Grosso', 'year': '1958'}]},
 {'_id': {'artist': 'Annie Ross & Gerry Mulligan'},
  'release': [{'title': 'Sings a Song With Mulligan', 'year': '1959'}]},
 {'_id': {'artist': 'Barron Mingus and his Rhythm'},
  'release': [{'title': "Pennies From Heaven / Lyon's Roar", 'year': '1949'}]},
 {'_id': {'artist': 'Baron Mingus and his Octet'},
  'release': [{'title': 'Make Believe / Bedspread', 'year': '1946'},
   {'title': 'Honey Take a Chance With Me / This Subdues My Passion',
    'year': '1946'}]},
 {'_id': {'artist': 'Charlie Mingus and orchestra / Roy Porter and Orchestra'},
  'release': [{'title': "The Story of Love / Don't Blame Me",
    'year': '1949'}]},
 {'_id': {'artist': 'Strings and Keys'},
  'release': [{'title': 'Strings and Keys', 'year': '1953'}]},
 {'_id': {'artist': 'Charles Mingus & John LaPorta'},
  'release': [{'title': 'Jazzical Moods, Volume 2', 'year': '1955'},
   {'title': '